In [ ]:
from quspin.basis import spin_basis_general # Hilbert space spin basis
import numpy as np
from Library import Sigma, chi_trunc
from numpy import linalg as LA
import matplotlib.pyplot as plt
from quspin.basis import spin_basis_1d 
from scipy.linalg import expm
import cProfile
import pstats

import numpy as np

data_type=np.complex128
S_z, S_x, S_y, _ = Sigma()

def Create_Hamiltonina(Model, S1, Interactions, Fields):
    H = np.zeros((d,d,d,d)).astype(np.complex128)

    for j in range(len(Model[0])):
      H += np.einsum("ij,kl->ikjl",Interactions[Model[0][j]]*S1[Model[0][j]],S1[Model[0][j]]) #two-site

    for k in range(len(Model[1])):
      H += np.einsum("ij,kl->ikjl",Fields[Model[1][k]]*S1[Model[1][k]],np.diag(np.ones(2)))    #on-site
      H += np.einsum("ij,kl->ikjl",np.diag(np.ones(2)),Fields[Model[1][k]]*S1[Model[1][k]])
    return H
    
def Compute_Theta(lam_0,Gamma_0,lam_1,Gamma_1):
    Theta=np.einsum("a,aib->aib",lam_0,Gamma_0)
    Theta=np.einsum("aib,b->aib",Theta,lam_1)
    Theta=np.einsum("aib,bjc->aijc",Theta,Gamma_1)
    Theta=np.einsum("aijc,c->aijc",Theta,lam_0)

    return Theta

def apply_two_site_unitary(lam_0,Gamma_0,lam_1,Gamma_1,U,chi):
    Theta = Compute_Theta(lam_0,Gamma_0,lam_1,Gamma_1)

    Theta=np.einsum("ijkl,aijb->aklb",U,Theta)
    Theta=Theta.reshape(chi*d,chi*d)         

    Sigma,lam_1,Ve=chi_trunc(chi,Theta)

    vi=np.divide(1.0, lam_0, out=np.zeros_like(lam_0), where=np.abs(lam_0)>=1E-14)
    Gamma_1=np.einsum("aib,b->aib",Ve,vi)                            
    Gamma_0=np.einsum("a,aib->aib",vi,Sigma) 
    
    return Gamma_0,Gamma_1,lam_1

def Exp_value_2_site(lam[0],Gamma[0],lam[1],Gamma[1],S_1,S_2):
   Theta = Compute_Theta(lam[0],Gamma[0],lam[1],Gamma[1])
   
   Mz=np.einsum("ik,jm->ikjm",S_1 ,S_2 )
   Mz=np.einsum("aijb,ikjm->akmb",Theta,Mz)
   Mz=np.einsum("akmb,akmb->",Mz,np.conj(Theta))
   return Mz

def iTEBD():
  for ty in range(N):
   # Evolve the system imposing PBC
   Gamma[0],Gamma[1],lam[1]=apply_two_site_unitary(lam[0],Gamma[0],lam[1],Gamma[1],U,chi)
   Gamma[1],Gamma[0],lam[0]=apply_two_site_unitary(lam[1],Gamma[1],lam[0],Gamma[0],U,chi)
   
   # Calculate magnetization in the Z direction
   Mz_iTEBD[ty+1] = Exp_value_2_site(lam[0],Gamma[0],lam[1],Gamma[1],S_z,S_z)

In [ ]:
T = 2.5*2    # total time
N = 75*6     # Number of updates
dt = T/N       # time step

chi = 20              # Bond dimension
d = 2                                  # Dimension of physical index
h_x, h_y, h_z = 0.5, 0.9, 0.5  # Field in the different directions
J_xx, J_yy, J_zz = 1.5, 0.2, 1.5  # Strength of spin interactions
Model = [["x"],[]]     # The first part of the model gives us the spin interaction and the second one the fields

# Initial antiferromagnetic product state
Gamma_0 = np.zeros((chi,d,chi)).astype(complex)
Gamma_0[0,0,0]=1.+0j
Gamma_0/=LA.norm(Gamma_0)

Gamma_1=np.zeros((chi,d,chi)).astype(complex)
Gamma_1[0,1,0]=1.+0j
Gamma_1/=LA.norm(Gamma_1)

Gamma=[Gamma_0,Gamma_1]

lama=np.zeros(chi)
lama[0]=1.
lam=[lama,lama]

# Hamiltonian
H  = Create_Hamiltonina(Model,S_z,Interactions,Fields)

# Time evolution operator
U=expm(-1j*dt*H.reshape(d**2,-1)).reshape(d,d,d,d)
# 2 site wave funciotn

Mz_iTEBD = np.zeros(N+1).astype(data_type)
Mz_iTEBD[0] = Exp_value_2_site(lam[0],Gamma[0],lam[1],Gamma[1],S_z,S_z)

# Profile and Execute function
profiler = cProfile.Profile()
profiler.enable()

iTEBD()              # Main code

profiler.disable()
stats = pstats.Stats(profiler).sort_stats('tottime')
stats.print_stats()

#Plotting 
plt.plot(Mz_iTEBD, '-b',label="Split - expectation value")
plt.legend()
plt.xlabel("Number of time steps")
plt.ylabel("Expectation value")
plt.show()

In [ ]:


from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.tools.evolution import expm_multiply_parallel # expm_multiply_parallel
#  Quspin evolution

L=16   # system size
np.random.seed(0)   # random seed
basis = spin_basis_1d(L,pauli=1,)     # Hilbert space                               


Jzz=0.29        # zz interaction
Jxx=0.           # xx interaction
hx=np.sqrt(3)*2               # x interaction
hy=np.log(2)*2                # y interaction
                
J_zz  = [[Jzz,i,(i+1)%L] for i in range(L)] 
J_xx  = [[Jxx,i,(i+1)%L] for i in range(L)]
J_zz1 = [[1.,i,(i+1)%L] for i in range(L)]


h_x=[[hx,i] for i in range(L)]
h_y=[[hy,i] for i in range(L)]

static_H = [["xx",J_xx],["zz",J_zz],["x",h_x],["y",h_y]] #HAMILTONIAN
static_O = [["zz",J_zz1]]       # Observable
dynamic=[]


Fs=antiferomagnets.astype(complex)
#Fs = np.ones(2**L).astype(complex)
#Fs/=LA.norm(Fs)

H = hamiltonian(static_H,dynamic,basis=basis)   #HAMILTONIAN
O = hamiltonian(static_O,dynamic,basis=basis)   #OBSERVABLE

expectation_O = np.zeros(N+1,dtype=np.float64)
expectation_O[0] = O.expt_value(Fs).real

exp_H = expm_multiply_parallel(H.tocsr(),a=-1j*dt)  #Time evolution operator
for d in range(N):

  exp_H.dot(Fs,overwrite_v=True)    #evolve
  expectation_O[d+1] = O.expt_value(Fs).real   #measure observable
  
#Plotting

plt.plot(expectation_O/(L),label="ZZ- expectation value")

plt.legend()
plt.xlabel("Number of time steps")
plt.ylabel("Expectation value")
plt.show()
Hermiticity check passed!
Symmetry checks passed!
Hermiticity check passed!
Symmetry checks passed!

#Checking the observables

plt.plot(expectation_O/L-Energy_iTEBD,'r',label="Difference in the expectation value of Observables")
plt.legend()
plt.xlabel("Number of steps")
plt.ylabel("Difference in expectation value")
plt.show()
E:\Anaconda\lib\site-packages\numpy\core\_asarray.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
